In [ ]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import SAGEConv
import copy
import torch
from torch.utils.data import DataLoader, Dataset
import csv
import json
import pandas as pd
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
torch.manual_seed(0)

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Building graph from the edge information

from tqdm import tqdm
import networkx as nx

node_list_exp = []

G = nx.Graph()
with open('/content/drive/MyDrive/GSPPI/Edge.csv', 'r') as f:
    data = csv.reader(f)
    # headers = next(data)
    for row in tqdm(data):
        node_list_exp.append(row[0])
        node_list_exp.append(row[1])

print('len',len(node_list_exp))
set_list = set(node_list_exp)
list_set = list(set_list)
print(len(list_set))


13163it [00:00, 16029.19it/s]

len 26326
5497


In [ ]:
excel_file = "/content/drive/MyDrive/GSPPI/prot_id_data.xlsx"
df_prot_id = pd.read_excel(excel_file)

# Create the dictionary from the DataFrame
key_value_dict = {}
for index, row in df_prot_id.iterrows():
    key = row['Key']
    value = row['Value']
    key_value_dict[key] = value

# Display the recreated dictionary
print(key_value_dict)

{'Q7L590': 0, 'Q96A72': 1, 'P52657': 2, 'Q14103': 3, 'Q05519': 4, 'Q06418': 5, 'Q13507': 6, 'O00418': 7, 'Q9Y6Y9': 8, 'Q92990': 9, 'Q99731': 10, 'Q68CZ2': 11, 'Q9UGI8': 12, 'P48645': 13, 'Q92900': 14, 'Q9UNY4': 15, 'Q9NQT4': 16, 'P07384': 17, 'O95352': 18, 'P49589': 19, 'Q07954': 20, 'Q00722': 21, 'Q6IE81': 22, 'P36507': 23, 'Q96J92': 24, 'P39748': 25, 'Q9NNX6': 26, 'Q9UGK8': 27, 'O15550': 28, 'P61457': 29, 'Q8N6T3': 30, 'P05109': 31, 'P98170': 32, 'Q9UKC9': 33, 'O43865': 34, 'O96019': 35, 'P48681': 36, 'P84077': 37, 'O14654': 38, 'P78559': 39, 'Q13509': 40, 'Q05513': 41, 'Q9H553': 42, 'Q15022': 43, 'P07947': 44, 'O00253': 45, 'O60447': 46, 'P35241': 47, 'Q86VP1': 48, 'P43005': 49, 'Q7Z6E9': 50, 'Q16548': 51, 'O60941': 52, 'P25788': 53, 'O15287': 54, 'P06748': 55, 'Q8TD31': 56, 'Q9H4M3': 57, 'O14879': 58, 'Q15700': 59, 'P60228': 60, 'Q14683': 61, 'Q5VSY0': 62, 'Q9Y6K9': 63, 'P36957': 64, 'P16157': 65, 'Q8N448': 66, 'P08908': 67, 'P63027': 68, 'P08865': 69, 'Q15562': 70, 'Q96EK4': 71, '

In [ ]:
# Building graph from the edge information

from tqdm import tqdm
import networkx as nx

node_list_1 = []
node_list_2 = []
G = nx.Graph()
with open('/content/drive/MyDrive/GSPPI/Edge.csv', 'r') as f:
    data = csv.reader(f)
    #headers = next(data)
    for row in tqdm(data):
        G.add_node(key_value_dict[row[0]]) #protein1 in first column
        G.add_node(key_value_dict[row[1]]) #protein2 in second column
        node_list_1.append(key_value_dict[row[0]])
        node_list_2.append(key_value_dict[row[1]])
        if G.has_edge(key_value_dict[row[0]], key_value_dict[row[1]]):
            # edge already exists, increase weight by one
            G[key_value_dict[row[0]]][key_value_dict[row[1]]]['weight'] += 1
        else:
            # add new edge with weight 1
            G.add_edge(key_value_dict[row[0]], key_value_dict[row[1]], weight = 1)

G_nodes = G.number_of_nodes()
G_edges = G.number_of_edges()
print("Nodes = ", G_nodes, " Edges = ",G_edges )

13163it [00:00, 149645.94it/s]

Nodes =  5497  Edges =  13163


In [ ]:
edges = [(source, target) for source, target in G.edges]

# Convert the edge index to a PyTorch tensor with shape [2, num_edges]
edge_index = torch.tensor(edges, dtype=torch.long).t()
print(edge_index)
print(edge_index.shape)

tensor([[5077, 5077, 5077,  ..., 4002, 1072, 3744],
        [3396, 4729,  414,  ..., 1376, 4631,  965]])
torch.Size([2, 13163])


In [ ]:
# combine all nodes in a list
node_list = node_list_1 + node_list_2
print(node_list)

# remove duplicate items from the list
node_list = list(dict.fromkeys(node_list))
print(node_list)

# build adjacency matrix
adj_G = nx.to_numpy_array(G, nodelist = node_list)
#adj_G.shape
#print(adj_G)


[5077, 592, 4283, 5019, 3954, 4128, 3247, 637, 2754, 3179, 241, 2392, 3065, 2238, 1948, 377, 3041, 4728, 4202, 866, 2461, 3476, 1722, 1256, 1892, 5223, 3170, 2754, 2900, 4206, 3274, 448, 4607, 4729, 1600, 5077, 4261, 2001, 4283, 5398, 4920, 4859, 4165, 1139, 2392, 2898, 3195, 4142, 2240, 3296, 1045, 1734, 4865, 1291, 453, 3965, 261, 2308, 4379, 2514, 4488, 2172, 637, 3379, 3274, 5223, 5388, 2301, 5223, 4384, 5223, 906, 103, 5133, 5223, 1239, 2430, 208, 2927, 3813, 2940, 1712, 3868, 4749, 4171, 1916, 355, 4698, 1892, 4725, 3274, 2466, 3115, 4478, 1699, 3855, 5067, 3705, 235, 4539, 3981, 2556, 4575, 372, 193, 275, 1906, 3788, 351, 799, 3886, 855, 4213, 3845, 1872, 1699, 3705, 4575, 3886, 1699, 3705, 193, 3845, 5079, 3790, 5319, 1970, 3969, 5176, 3807, 684, 4725, 4839, 3969, 459, 5176, 3807, 1872, 3855, 4575, 501, 2754, 467, 108, 2683, 1179, 1154, 2392, 2514, 906, 127, 2238, 3041, 2001, 2987, 4874, 4283, 2855, 580, 5428, 702, 4622, 1625, 4604, 2005, 1483, 669, 5169, 95, 2403, 1501, 4157, 

In [ ]:
tensor_adj = torch.from_numpy(adj_G )
tensor_adj = tensor_adj.to(torch.long)
print(tensor_adj)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [ ]:
from tqdm import tqdm
import networkx as nx

In [ ]:
pos_train_list = []
with open('/content/drive/MyDrive/GSPPI/train_edge_pos_data.csv', 'r') as f:
    data = csv.reader(f)
    headers = next(data)
    for row in tqdm(data):
        tuple = (key_value_dict[row[0]], key_value_dict[row[1]])
        pos_train_list.append(tuple)

train_pos_edge = torch.tensor(pos_train_list, dtype=torch.long)
print(train_pos_edge)

10530it [00:00, 379870.48it/s]

tensor([[5077, 3396],
        [5077, 4729],
        [5077,  414],
        ...,
        [5189, 3103],
        [3475, 1570],
        [3475, 4991]])


In [ ]:
neg_train_list = []
with open('/content/drive/MyDrive/GSPPI/train_edge_neg_data.csv', 'r') as f:
    data = csv.reader(f)
    headers = next(data)
    for row in tqdm(data):
        tuple = (key_value_dict[row[0]], key_value_dict[row[1]])
        neg_train_list.append(tuple)

train_neg_edge = torch.tensor(neg_train_list, dtype=torch.long)
print(train_neg_edge)

10530it [00:00, 324444.80it/s]

tensor([[ 511, 2946],
        [3836, 3453],
        [4625, 2251],
        ...,
        [5303, 2494],
        [1070, 2654],
        [2444, 3233]])


In [ ]:
pos_test_list = []
with open('/content/drive/MyDrive/GSPPI/test_edge_pos_data.csv', 'r') as f:
    data = csv.reader(f)
    headers = next(data)
    for row in tqdm(data):
        tuple = (key_value_dict[row[0]], key_value_dict[row[1]])
        pos_test_list.append(tuple)

test_pos_edge = torch.tensor(pos_test_list, dtype=torch.long)
print(test_pos_edge)

2633it [00:00, 301993.56it/s]

tensor([[3475, 1345],
        [3475,  268],
        [3475, 2643],
        ...,
        [4002, 1376],
        [1072, 4631],
        [3744,  965]])


In [ ]:
neg_test_list = []
with open('/content/drive/MyDrive/GSPPI/test_edge_neg_data.csv', 'r') as f:
    data = csv.reader(f)
    headers = next(data)
    for row in tqdm(data):
        tuple = (key_value_dict[row[0]], key_value_dict[row[1]])
        neg_test_list.append(tuple)

test_neg_edge = torch.tensor(neg_test_list, dtype=torch.long)
print(test_neg_edge)

2633it [00:00, 378789.31it/s]

tensor([[ 836, 2524],
        [1051, 4238],
        [5468, 4558],
        ...,
        [2807, 5142],
        [ 282, 3898],
        [2109, 5387]])


In [ ]:
Dict = { }
Dict['train'] = {'edge':train_pos_edge , 'edge_neg': train_neg_edge}
Dict['test'] = {'edge':test_pos_edge , 'edge_neg': test_neg_edge}
print(Dict)

{'train': {'edge': tensor([[5077, 3396],
        [5077, 4729],
        [5077,  414],
        ...,
        [5189, 3103],
        [3475, 1570],
        [3475, 4991]]), 'edge_neg': tensor([[ 511, 2946],
        [3836, 3453],
        [4625, 2251],
        ...,
        [5303, 2494],
        [1070, 2654],
        [2444, 3233]])}, 'test': {'edge': tensor([[3475, 1345],
        [3475,  268],
        [3475, 2643],
        ...,
        [4002, 1376],
        [1072, 4631],
        [3744,  965]]), 'edge_neg': tensor([[ 836, 2524],
        [1051, 4238],
        [5468, 4558],
        ...,
        [2807, 5142],
        [ 282, 3898],
        [2109, 5387]])}}


In [ ]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.leaky_relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)

        return x

In [ ]:
class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            # x = F.mish(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)

pfeatures

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif , chi2

In [ ]:
feature_location = '/content/drive/MyDrive/GSPPI/pfeature.csv'
columns_to_skip = 1
feature_df = pd.read_csv(feature_location) # , usecols=range(columns_to_skip)
feature_df = feature_df.iloc[:, columns_to_skip:]
feature_df
feature_df = feature_df.dropna(axis=1)
feature_df.shape

(5497, 976)

In [ ]:
from sklearn.feature_selection import VarianceThreshold
var_thres = VarianceThreshold(threshold = 0.08)
var_thres.fit(feature_df)

VarianceThreshold(threshold=0.08)

In [ ]:
constant_columns = [column for column in feature_df.columns
                    if column not in feature_df.columns[var_thres.get_support()]]
print(len(constant_columns))
features_new = feature_df.drop(constant_columns , axis = 1)
features_new.shape[1]

690


286

In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=200)  # You can adjust the number of components as needed
# features_new = pca.fit_transform(features_new)

In [ ]:
df_transformed = pd.DataFrame(features_new)
df_transformed

,AAC_A,AAC_C,AAC_D,AAC_E,AAC_F,AAC_G,AAC_H,AAC_I,AAC_K,AAC_L,...,QSO1_SC_L,QSO1_SC_N,QSO1_SC_P,QSO1_SC_Q,QSO1_SC_R,QSO1_SC_S,QSO1_SC_T,QSO1_SC_V,QSO1_SC_Y,SOC1_G1
0,6.74,2.29,4.00,10.29,2.74,5.49,1.26,3.66,9.94,9.49,...,2.5089,0.8766,1.6323,1.3905,1.7230,2.1159,1.6323,1.1789,0.3930,9392.7551
1,3.38,0.68,8.11,8.11,6.08,6.08,3.38,8.11,8.78,8.78,...,1.7986,0.6918,0.8301,0.4151,1.1068,1.2452,0.4151,1.5219,1.1068,9955.2517
2,6.42,1.83,5.50,5.50,4.59,3.67,0.00,4.59,4.59,11.93,...,2.6225,2.0173,0.2017,1.8156,1.4121,1.0087,1.8156,2.0173,0.6052,10291.8981
3,8.17,0.85,4.23,8.45,3.94,16.34,1.41,2.82,9.01,2.54,...,0.6386,1.1354,0.7096,1.2773,0.9934,1.9869,1.0644,1.0644,1.4902,9720.9153
4,6.20,0.41,6.20,7.44,1.24,7.23,2.07,1.86,10.74,6.61,...,2.0138,0.5664,2.2026,0.5664,3.9018,4.2794,1.5104,1.5733,0.2517,7702.0849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5492,9.65,3.06,4.12,4.82,1.76,7.18,2.35,2.24,4.47,7.29,...,2.1074,0.8837,2.7192,1.1557,2.5492,3.9428,1.2576,1.3936,0.9177,9279.9270
5493,5.14,0.91,4.53,7.25,4.53,3.02,3.32,6.95,6.95,10.57,...,2.7607,1.1043,1.1043,0.7099,1.0254,2.2086,1.6564,2.8396,1.0254,9353.2515
5494,7.37,1.60,4.49,9.62,1.28,6.41,1.28,7.37,11.86,7.69,...,2.0178,1.0930,1.4293,1.5975,0.9248,1.5975,1.0930,2.0178,0.2522,9337.5209
5495,10.46,0.74,6.19,7.81,2.80,8.25,1.18,5.74,7.66,7.07,...,1.9024,1.0701,0.8719,1.3872,1.4268,1.6250,1.6646,2.0609,0.4360,9245.4499


In [ ]:
feature_columns = df_transformed.columns
# print(feature_columns)
features_new = df_transformed[feature_columns].values
print(features_new)

[[6.74000000e+00 2.29000000e+00 4.00000000e+00 ... 1.17890000e+00
  3.93000000e-01 9.39275510e+03]
 [3.38000000e+00 6.80000000e-01 8.11000000e+00 ... 1.52190000e+00
  1.10680000e+00 9.95525170e+03]
 [6.42000000e+00 1.83000000e+00 5.50000000e+00 ... 2.01730000e+00
  6.05200000e-01 1.02918981e+04]
 ...
 [7.37000000e+00 1.60000000e+00 4.49000000e+00 ... 2.01780000e+00
  2.52200000e-01 9.33752090e+03]
 [1.04600000e+01 7.40000000e-01 6.19000000e+00 ... 2.06090000e+00
  4.36000000e-01 9.24544990e+03]
 [6.06000000e+00 2.22000000e+00 7.53000000e+00 ... 2.01730000e+00
  6.85900000e-01 9.89898080e+03]]


In [ ]:
# selector = SelectKBest(chi2, k=2)
# X_new = selector.fit_transform(features_new , y = None)# .transform(features_new)
# print(X_new)

In [ ]:
# Convert the node features from numpy array to a PyTorch tensor
x_new = torch.tensor(features_new, dtype=torch.float) # .to_numpy()
x_new= torch.nn.functional.normalize(x_new)
min_vals, _ = torch.min(x_new, dim=0)
max_vals, _ = torch.max(x_new, dim=0)
normalized_data = (x_new - min_vals) / (max_vals - min_vals)
normalized_df = pd.DataFrame(normalized_data.numpy(), columns=df_transformed.columns)
x_pfeatures = torch.tensor(normalized_df.values, dtype=torch.float)
# print(x_new)
print(x_pfeatures.shape)

torch.Size([5497, 286])


In [ ]:
data = Data(num_nodes = G_nodes, x=x_pfeatures, adj_t = edge_index ,edge_index = edge_index )
print(data)

Data(x=[5497, 286], edge_index=[2, 13163], num_nodes=5497, adj_t=[2, 13163])


In [ ]:
hidden_channels = 256
out_channels = 128
num_layers_model = 3
num_layers_predictor = 2
dropout = 0.003
batch_size = 64 * 1024
lr = 0.002
weight_decay=1e-4
input_dim = features_new.shape[1]

In [ ]:
class LinkGNN(torch.nn.Module):
    def __init__(self, gnn, linkpred):
        super(LinkGNN, self).__init__()
        self.gnn = gnn
        self.linkpred = linkpred

    def reset_parameters(self):
        self.gnn.reset_parameters()
        self.linkpred.reset_parameters()

    def forward(self,x , edges, adj):
        h = self.gnn(x, adj)
        return self.linkpred(h[edges[0]], h[edges[1]])

In [ ]:
def build_model(data):

        gnn = SAGE(
            input_dim, hidden_channels,
            hidden_channels, num_layers_model,
            dropout)

        linkpred = LinkPredictor(
            hidden_channels, out_channels,
            1, num_layers_predictor,
            dropout)
        model = LinkGNN(gnn, linkpred)

        return model

In [ ]:
model = build_model(data)

In [ ]:
optimizer = torch.optim.Adam(list(model.parameters()) , lr=lr , weight_decay=weight_decay)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
def accuracy(pos_out , neg_out):
    threshold = 0.5  # Example threshold for classification

    # Convert prediction scores to binary predictions using the threshold
    pos_predictions = (pos_out > threshold).long()
    neg_predictions = (neg_out > threshold).long()
    # print('neg_predictions--' , neg_predictions)

    # Create binary ground truth labels for positive and negative edges
    pos_labels = torch.ones_like(pos_predictions)
    neg_labels = torch.zeros_like(neg_predictions)

    # Concatenate predictions and labels for positive and negative edges
    all_predictions = torch.cat((pos_predictions, neg_predictions))
    all_labels = torch.cat((pos_labels , neg_labels))

    # Calculate accuracy
    accuracy = (all_predictions == all_labels).float().mean().item()
    # accuracy = (pos_predictions == pos_labels).float().mean().item()
    roc_auc = roc_auc_score(all_labels, all_predictions)

    return accuracy , roc_auc


In [ ]:
def metrics(pos_out , neg_out):
    threshold = 0.5  # Example threshold for classification

    # Convert prediction scores to binary predictions using the threshold
    pos_predictions = (pos_out > threshold).long()
    neg_predictions = (neg_out > threshold).long()

    # Create binary ground truth labels for positive and negative edges
    pos_labels = torch.ones_like(pos_predictions)
    neg_labels = torch.zeros_like(neg_predictions)

    # Concatenate predictions and labels for positive and negative edges
    all_predictions = torch.cat((pos_predictions, neg_predictions))
    all_labels = torch.cat((pos_labels, neg_labels))

    # Calculate true positives (TP), false positives (FP), true negatives (TN), false negatives (FN)
    TP = ((all_predictions == 1) & (all_labels == 1)).sum().item()
    FP = ((all_predictions == 1) & (all_labels == 0)).sum().item()
    TN = ((all_predictions == 0) & (all_labels == 0)).sum().item()
    FN = ((all_predictions == 0) & (all_labels == 1)).sum().item()


    # Calculate precision and recall
    precision = TP / (TP + FP) if TP + FP > 0 else 0.0
    recall = TP / (TP + FN) if TP + FN > 0 else 0.0
    if TN + FN > 0 and FP + TP > 0 and TN + FP > 0 and FN + TP > 0:
        mcc_met = ((TP * TN) - (FP * FN)) / math.sqrt((TN + FN) * (FP + TP) * (TN + FP) * (FN + TP))
    else:
        mcc_met = 0.0

    confusion_matrix = np.array([[TP, FP], [FN, TN]])
    class_names = ['Positive', 'Negative']
    # plot_confusion_matrix(confusion_matrix, class_names)

    return precision , recall , mcc_met

In [ ]:
def plot_confusion_matrix(confusion_matrix, class_names):
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)

    thresh = confusion_matrix.max() / 2.0
    # for i, j in np.ndenumerate(confusion_matrix):
    #     plt.text(j, i, f'{confusion_matrix[i, j]}', horizontalalignment='center', color='white' if confusion_matrix[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()



In [ ]:
from torch_geometric.utils import negative_sampling, to_undirected

In [ ]:
def train(model, data, split_edge, optimizer, batch_size , epoch):
    model.train()
    # predictor.train()

    total_loss = total_examples = 0
    pos_train_preds = []
    neg_train_preds = []
    pos_train_edge = split_edge['train']['edge']#.to(data.x)#.device)
    neg_train_edge = split_edge['train']['edge_neg']#.to(data.x)#.device)

    # scheduler = MultiStepLR(optimizer, milestones=[1,5], gamma=0.5)
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size, shuffle=True):
        #print('perm--', perm)
        optimizer.zero_grad()
        pos_edge = pos_train_edge[perm].t()
        neg_edge = neg_train_edge[perm].t()
        # neg_edge = negative_sampling(edge_index, num_nodes=data.num_nodes, num_neg_samples=pos_edge.size(1), method='dense')
        out = model( data.x, torch.cat([pos_edge, neg_edge], 1), data.adj_t).squeeze()

        pos_out = out[:pos_edge.size(1)]
        pos_loss = -torch.log(pos_out + 1e-8).mean()

        neg_out = out[pos_edge.size(1):]
        neg_loss = -torch.log(1 - neg_out + 1e-8).mean()

        loss = pos_loss + neg_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        #accuracy
        train_accuracy , train_rocauc = accuracy(pos_out , neg_out)
        train_precision , train_recall , train_mcc = metrics(pos_out , neg_out)
        f1_score = 2 * (train_precision * train_recall) / (train_precision + train_recall) if (train_precision + train_recall) > 0 else 0.0

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples
        avg_loss_train = total_loss / total_examples

        print(f'Epoch: {epoch:02d}, ', f'Train loss: {avg_loss_train:.4f}, ',  f'Train accuracy: {train_accuracy:.4f}, '
              f'Train precision: {train_precision:.4f}, ' , f'Train recall: {train_recall:.4f}, ' , f'Train mcc: {train_mcc:.4f}, '
              , f'Train f1_score: {f1_score:.4f}, ' ,f'Train rocauc: {train_rocauc:.4f}, ' )
    # scheduler.step()

    return train_accuracy , total_loss / total_examples

In [ ]:
@torch.no_grad()
def test(model, data, split_edge, evaluator, batch_size):
    model.eval()

    pos_test_edge = split_edge['test']['edge']#.to(data.x)#.device)
    neg_test_edge = split_edge['test']['edge_neg']#.to(data.x)#.device)

    total_loss = total_examples = 0
    test_accuracy = test_precision = test_recall = 0.0
    #pos_test_preds , neg_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size, shuffle=False):

        pos_edge = pos_test_edge[perm].t()
        neg_edge = neg_test_edge[perm].t()
        # neg_edge = negative_sampling(edge_index, num_nodes=data.num_nodes, num_neg_samples=pos_edge.size(1), method='dense')
        out = model( data.x, torch.cat([pos_edge, neg_edge], 1), data.adj_t).squeeze()

        pos_out = out[:pos_edge.size(1)]
        pos_loss = -torch.log(pos_out + 1e-8).mean()

        neg_out = out[pos_edge.size(1):]
        neg_loss = -torch.log(1 - neg_out + 1e-8).mean()

        loss = pos_loss + neg_loss
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #accuracy
        test_accuracy ,test_rocauc = accuracy(pos_out , neg_out)
        test_precision , test_recall , test_mcc = metrics(pos_out , neg_out)
        f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall) if (test_precision + test_recall) > 0 else 0.0

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

        print(f'Test accuracy: {test_accuracy:.4f}, '
              f'Test precision: {test_precision:.4f}, ' , f'Test recall: {test_recall:.4f}, ' , f'Test mcc: {test_mcc:.4f}, ' ,
              f'Test f1_score: {f1_score:.4f}, ' , f'Test rocauc: {test_rocauc:.4f}, ' )

    return test_accuracy , total_loss / total_examples

In [ ]:
data

Data(x=[5497, 286], edge_index=[2, 13163], num_nodes=5497, adj_t=[2, 13163])

In [ ]:
class ModelCheckpoint:
    def __init__(self, model_save_load_dir ):
        self.model_dir = model_save_load_dir
        self.num_epochs = 0
        self.metric_arr = []
        self.saved_at = ""
        self.model_state_dict_key = 'model_state_dict'
        self.optimizer_state_dict_key = 'optimizer_state_dict'
        self.metric_dict_key = 'metric'
        self.num_epochs_key = 'num_epochs'
        self.metric = -np.inf
        self.model_state = None
        self.optim_state = None


    def get_metric_arr(self):
        return self.metric_arr

    def get_last_saved_loc(self):
        return self.saved_at

    def get_num_epochs(self):
        return self.num_epochs

    def load_checkpoint(self, model_load_file_name : str = "some.pth.tar", model = None, optim = None):
        self.saved_at = model_load_file_name
        checkpoint = torch.load(self.saved_at)
        self.num_epochs = checkpoint[self.num_epochs_key]
        self.metric = checkpoint[self.metric_dict_key]
        model.load_state_dict(checkpoint[self.model_state_dict_key])
        self.model_state = copy.deepcopy(model)
        if (optim != None):
            optim.load_state_dict(checkpoint[self.optimizer_state_dict_key])
            self.optim_state = copy.deepcopy(optim)

        print(f"Model loaded successfully from checkpoint with metric = {self.metric:.5f}, num_epochs = {self.num_epochs:03d}...")

    def save_checkpoint(self):
        print("=> Saving a new best")
        # save_file_name = f'{self.model_save_prefix}_epoch_{self.num_epochs:03d}_metric_{self.metric:.5f}.pth.tar'
        save_at = self.model_dir
        if (self.optim_state != None):
            state_dict = {self.num_epochs_key : self.num_epochs,
                        self.model_state_dict_key : self.model_state.state_dict(),
                        self.optimizer_state_dict_key : self.optim_state.state_dict(),
                        self.metric_dict_key : self.metric}
        else:
            state_dict = {self.num_epochs_key : self.num_epochs,
                        self.model_state_dict_key : self.model_state.state_dict(),
                        self.metric_dict_key : self.metric}
        torch.save(state_dict, save_at)
        self.saved_at = save_at
        print("Best model saved at location : ", save_at)


    def change_current(self, is_best, metric, state, optim):
        if is_best:
            self.metric = metric
            self.model_state = copy.deepcopy(state)
            if optim != None :
                self.optim_state = copy.deepcopy(optim)
            self.save_checkpoint()


    def create_checkpoint(self, state, metric, epoch, optim = None):
        self.num_epochs = epoch
        self.metric_arr.append(self.metric)
        self.change_current(True, metric, state, optim)


In [ ]:
min_loss_train = 100
best_accuracy_train = 0.0
min_loss_epoch_train = 80
best_acc_epoch_train = 0

min_loss_valid = 100
best_accuracy_valid = 0.0
min_loss_epoch_valid = 80
best_acc_epoch_valid = 0

epochs = 200
#early stopping
n_epochs_stop = 20
epochs_no_improve = 0
early_stop = False
model_save_path = '/content/drive/MyDrive/GSPPI_HIPPIE_WD/pfeature_0.08_hippie.pth.tar'
modelcheckpoint = ModelCheckpoint(model_save_load_dir=model_save_path)
for epoch in range(1, 1 + epochs):
    train_accuracy , train_loss = train(model, data, Dict, optimizer, batch_size , epoch)

    if(train_accuracy > best_accuracy_train):
      # print("Inside accuracy > best_accuracy")
      best_accuracy_train = train_accuracy
      best_acc_epoch_train = epoch
      # path to save the model
      modelcheckpoint.create_checkpoint(model, best_accuracy_train, epoch, optimizer)

    if(train_loss < min_loss_train):
      # print("loss< min_loss")
      epochs_no_improve = 0
      min_loss_train = train_loss
      min_loss_epoch_train = epoch
    elif train_loss >= min_loss_train :
      print("loss> min_loss")
      epochs_no_improve += 1

    if epoch > 5 and epochs_no_improve == n_epochs_stop:
      print('Early stopping!' )
      early_stop = True
      break

modelcheckpoint.create_checkpoint(model, best_accuracy_train, epoch, optimizer)

print(f'min_train_loss : {min_loss_train} for epoch {min_loss_epoch_train} ............... best_train_accuracy : {best_accuracy_train} for epoch {best_acc_epoch_train}')
print("Model saved")

Epoch: 01,  Train loss: 1.3863,  Train accuracy: 0.5000, Train precision: 0.0000,  Train recall: 0.0000,  Train mcc: 0.0000,  Train f1_score: 0.0000,  Train rocauc: 0.5000, 
=> Saving a new best
Best model saved at location :  /content/drive/MyDrive/GSPPI_HIPPIE_WD/pfeature_0.08_hippie.pth.tar
Epoch: 02,  Train loss: 1.3817,  Train accuracy: 0.5000, Train precision: 0.5000,  Train recall: 1.0000,  Train mcc: 0.0000,  Train f1_score: 0.6667,  Train rocauc: 0.5000, 
Epoch: 03,  Train loss: 1.3941,  Train accuracy: 0.4981, Train precision: 0.4990,  Train recall: 0.9794,  Train mcc: -0.0140,  Train f1_score: 0.6612,  Train rocauc: 0.4981, 
loss> min_loss
Epoch: 04,  Train loss: 1.3785,  Train accuracy: 0.6260, Train precision: 0.6041,  Train recall: 0.7311,  Train mcc: 0.2578,  Train f1_score: 0.6616,  Train rocauc: 0.6260, 
=> Saving a new best
Best model saved at location :  /content/drive/MyDrive/GSPPI_HIPPIE_WD/pfeature_0.08_hippie.pth.tar
Epoch: 05,  Train loss: 1.3852,  Train accurac

In [ ]:
test_accuracy , loss = test(model, data, Dict, optimizer, batch_size)

#print(f'min_val_loss : {min_loss} for epoch {min_loss_epoch} ............... best_val_accuracy : {best_accuracy} for epoch {best_acc_epoch}')
print('\n')

Test accuracy: 0.7651, Test precision: 0.8302,  Test recall: 0.6665,  Test mcc: 0.5408,  Test f1_score: 0.7394,  Test rocauc: 0.7651, 




In [ ]:
@torch.no_grad()
def test_new_model(model, data, split_edge, optimizer, batch_size):
    model.eval()

    pos_test_edge = split_edge['test']['edge']
    neg_test_edge = split_edge['test']['edge_neg']

    total_loss = total_examples = 0
    test_accuracy = test_precision = test_recall = 0.0
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size, shuffle=False):

        pos_edge = pos_test_edge[perm].t()
        neg_edge = neg_test_edge[perm].t()
        out = model( data.x, torch.cat([pos_edge, neg_edge], 1), data.adj_t).squeeze()

        pos_out = out[:pos_edge.size(1)]
        pos_loss = -torch.log(pos_out + 1e-8).mean()

        neg_out = out[pos_edge.size(1):]
        neg_loss = -torch.log(1 - neg_out + 1e-8).mean()

        loss = pos_loss + neg_loss
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #accuracy
        test_accuracy , test_rocauc = accuracy(pos_out , neg_out)
        test_precision, test_recall, test_mcc = metrics(pos_out , neg_out)

        if(test_precision + test_recall) > 0:
           f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)
        else:
           f1_score = 0.0

        num_examples = pos_out.size(0)
        total_loss = total_loss+loss.item() * num_examples
        total_examples= total_examples+num_examples
        avg_loss_test = total_loss / total_examples

        print('Evaluation on saved model')
        print( f'Test loss: {avg_loss_test:.4f}, ',  f'Test accuracy: {test_accuracy:.4f}, ' , f'Test precision: {test_precision:.4f}, ' , f'Test recall: {test_recall:.4f}, ' , f'Test mcc: {test_mcc:.4f}, ' , f'Test f1_score: {f1_score:.4f}, ' , f'Test rocauc_score: {test_rocauc:.4f}, '  )

    return total_loss / total_examples

In [ ]:
modelcheckpoint = ModelCheckpoint(model_save_path)
model = build_model(data)
modelcheckpoint.load_checkpoint(model_save_path , model, optimizer)
print(model)

Model loaded successfully from checkpoint with metric = 0.93822, num_epochs = 200...
LinkGNN(
  (gnn): SAGE(
    (convs): ModuleList(
      (0): SAGEConv(286, 256, aggr=mean)
      (1-2): 2 x SAGEConv(256, 256, aggr=mean)
    )
  )
  (linkpred): LinkPredictor(
    (lins): ModuleList(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): Linear(in_features=128, out_features=1, bias=True)
    )
  )
)


In [ ]:
loss = test_new_model(model, data, Dict, optimizer, batch_size)

Evaluation on saved model
Test loss: 1.3186,  Test accuracy: 0.7651,  Test precision: 0.8302,  Test recall: 0.6665,  Test mcc: 0.5408,  Test f1_score: 0.7394,  Test rocauc_score: 0.7651, 
